In [26]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
df_1 = pd.read_csv('tmdb_5000_credits.csv')
df_1.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [28]:
df_2 = pd.read_csv('tmdb_5000_movies.csv')
df_2.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [29]:
df_1 = df_1.drop(['title'], axis=1)
df_1.head()

,movie_id,cast,crew
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [30]:
df_1.rename(columns={"movie_id": "id"}, inplace=True)


merged_df = pd.merge(df_1, df_2, on="id", how="inner")
merged_df.head()

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,...,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [35]:
for id_ in merged_df.index:
    try:
        cast_list = json.loads(merged_df.at[id_, 'cast'])[0:3]  
        character_names = ", ".join([actor["name"] for actor in cast_list])  
        merged_df.at[id_, 'cast'] = character_names  
    except (TypeError, json.JSONDecodeError):
        merged_df.at[id_, 'cast'] = ""  

print(merged_df[['title', 'cast']])  


                                         title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4798                               El Mariachi   
4799                                 Newlyweds   
4800                 Signed, Sealed, Delivered   
4801                          Shanghai Calling   
4802                         My Date with Drew   

                                                  cast  
0       Sam Worthington, Zoe Saldana, Sigourney Weaver  
1          Johnny Depp, Orlando Bloom, Keira Knightley  
2           Daniel Craig, Christoph Waltz, Léa Seydoux  
3           Christian Bale, Michael Caine, Gary Oldman  
4         Taylor Kitsch, Lynn Collins, Samantha Morton  
...                                                ...  


In [36]:
merged_df.cast

0         Sam Worthington, Zoe Saldana, Sigourney Weaver
1            Johnny Depp, Orlando Bloom, Keira Knightley
2             Daniel Craig, Christoph Waltz, Léa Seydoux
3             Christian Bale, Michael Caine, Gary Oldman
4           Taylor Kitsch, Lynn Collins, Samantha Morton
                              ...                       
4798    Carlos Gallardo, Jaime de Hoyos, Peter Marquardt
4799          Edward Burns, Kerry Bishé, Marsha Dietlein
4800            Eric Mabius, Kristin Booth, Crystal Lowe
4801             Daniel Henney, Eliza Coupe, Bill Paxton
4802     Drew Barrymore, Brian Herzlinger, Corey Feldman
Name: cast, Length: 4803, dtype: object

In [37]:
for id_ in merged_df.index:
    try:
        genres_list = json.loads(merged_df.at[id_, 'genres'])  
        genres_name = ", ".join([actor["name"] for actor in genres_list])  
        merged_df.at[id_, 'genres'] = genres_name  
    except (TypeError, json.JSONDecodeError):
        merged_df.at[id_, 'genres'] = ""  

print(merged_df[['title', 'genres']]) 

                                         title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4798                               El Mariachi   
4799                                 Newlyweds   
4800                 Signed, Sealed, Delivered   
4801                          Shanghai Calling   
4802                         My Date with Drew   

                                           genres  
0     Action, Adventure, Fantasy, Science Fiction  
1                      Adventure, Fantasy, Action  
2                        Action, Adventure, Crime  
3                  Action, Crime, Drama, Thriller  
4              Action, Adventure, Science Fiction  
...                                           ...  
4798                      Action, C

In [38]:
merged_df.genres

0       Action, Adventure, Fantasy, Science Fiction
1                        Adventure, Fantasy, Action
2                          Action, Adventure, Crime
3                    Action, Crime, Drama, Thriller
4                Action, Adventure, Science Fiction
                           ...                     
4798                        Action, Crime, Thriller
4799                                Comedy, Romance
4800               Comedy, Drama, Romance, TV Movie
4801                                               
4802                                    Documentary
Name: genres, Length: 4803, dtype: object

In [45]:
merged_df.loc[0].crew

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [46]:
data = json.loads(merged_df.loc[0].crew)
# Extract Director(s)
directors = [person["name"] for person in data if person["job"] == "Director"]

# Print the Director(s)
print("Director(s):", directors)

Director(s): ['James Cameron']


In [48]:
import json

# Iterate over each row to extract the director's name
for id_ in merged_df.index:
    try:
        crew_list = json.loads(merged_df.at[id_, 'crew'])  
        director = [person["name"] for person in crew_list if person["job"] == "Director"]
        merged_df.at[id_, 'director'] = ", ".join(director)  # Convert list to string
    except (TypeError, json.JSONDecodeError):
        merged_df.at[id_, 'director'] = ""  # Handle missing/invalid JSON

# Display the title and director columns
print(merged_df[['title', 'director']])


                                         title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4798                               El Mariachi   
4799                                 Newlyweds   
4800                 Signed, Sealed, Delivered   
4801                          Shanghai Calling   
4802                         My Date with Drew   

                                    director  
0                              James Cameron  
1                             Gore Verbinski  
2                                 Sam Mendes  
3                          Christopher Nolan  
4                             Andrew Stanton  
...                                      ...  
4798                        Robert Rodriguez  
4799                   

In [49]:
merged_df.head()

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,director
0,19995,"Sam Worthington, Zoe Saldana, Sigourney Weaver","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"Action, Adventure, Fantasy, Science Fiction",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,James Cameron
1,285,"Johnny Depp, Orlando Bloom, Keira Knightley","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"Adventure, Fantasy, Action",http://disney.go.com/disneypictures/pirates/,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Gore Verbinski
2,206647,"Daniel Craig, Christoph Waltz, Léa Seydoux","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"Action, Adventure, Crime",http://www.sonypictures.com/movies/spectre/,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,...,2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Sam Mendes
3,49026,"Christian Bale, Michael Caine, Gary Oldman","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"Action, Crime, Drama, Thriller",http://www.thedarkknightrises.com/,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,...,2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christopher Nolan
4,49529,"Taylor Kitsch, Lynn Collins, Samantha Morton","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"Action, Adventure, Science Fiction",http://movies.disney.com/john-carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",...,2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Andrew Stanton


In [54]:
### Applying vectoriztion on overview, cast, genre and director
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    4803 non-null   int64  
 1   cast                  4803 non-null   object 
 2   crew                  4803 non-null   object 
 3   budget                4803 non-null   int64  
 4   genres                4803 non-null   object 
 5   homepage              1712 non-null   object 
 6   keywords              4803 non-null   object 
 7   original_language     4803 non-null   object 
 8   original_title        4803 non-null   object 
 9   overview              4803 non-null   object 
 10  popularity            4803 non-null   float64
 11  production_companies  4803 non-null   object 
 12  production_countries  4803 non-null   object 
 13  release_date          4802 non-null   object 
 14  revenue               4803 non-null   int64  
 15  runtime              

In [53]:
merged_df[['overview', 'cast', 'genres', 'director']] = merged_df[['overview', 'cast', 'genres', 'director']].fillna("")


In [55]:
l = ['overview', 'cast', 'genres', 'director']
tfidf =  TfidfVectorizer(stop_words='english')

In [59]:
tfidf_list = []
for col in l:
    t = tfidf.fit_transform(merged_df[col])
    tfidf_list.append(t)

In [61]:
cos_sim = []
for i, col in enumerate(l):
    c = cosine_similarity(tfidf_list[i], tfidf_list[i])
    cos_sim.append(c)

In [75]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd  

def recommend_movies(title, merged_df, cos_sim, weights=[0.5, 0.3, 0.2, 0.2], top_n=10):
    title_col = "title_x" if "title_x" in merged_df.columns else "title"
    
    idx = merged_df[merged_df[title_col].str.lower() == title.lower()].index
    if len(idx) == 0:
        return pd.DataFrame(columns=[title_col, 'cast', 'genres', 'director'])  
    
    idx = idx[0]  
    scaler = MinMaxScaler()
    
    total_sim = np.array((weights[0] * cos_sim[0][idx]) + 
                          (weights[1] * cos_sim[1][idx]) + 
                          (weights[2] * cos_sim[2][idx]) + 
                          (weights[3] * cos_sim[3][idx])).reshape(-1, 1)
    
    print(total_sim)
    total_sim = scaler.fit_transform(total_sim).flatten()  
    
    sim_scores = list(enumerate(total_sim))
    
    
    sim_scores = [(i, score) for i, score in sim_scores if i != idx]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:top_n]  
    
    movie_indices = [i[0] for i in sim_scores]

    return merged_df[[title_col, 'cast', 'genres', 'director']].iloc[movie_indices]

print(recommend_movies("Inception", merged_df, cos_sim))


[[0.13890365]
 [0.07982517]
 [0.08015239]
 ...
 [0.00418388]
 [0.00413178]
 [0.00418996]]
                                   title  \
95                          Interstellar   
3573                             Memento   
1196                        The Prestige   
1568                              Looper   
1033                            Insomnia   
3                  The Dark Knight Rises   
65                       The Dark Knight   
134   Mission: Impossible - Rogue Nation   
119                        Batman Begins   
2897                              Cypher   

                                                   cast  \
95    Matthew McConaughey, Jessica Chastain, Anne Ha...   
3573       Guy Pearce, Carrie-Anne Moss, Joe Pantoliano   
1196        Hugh Jackman, Christian Bale, Michael Caine   
1568    Joseph Gordon-Levitt, Bruce Willis, Emily Blunt   
1033            Al Pacino, Robin Williams, Hilary Swank   
3            Christian Bale, Michael Caine, Gary Oldman   
65          

In [76]:
!pip install streamlit 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 624.5 kB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 473.0 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 KB 1.5 MB/s eta 0:00:001.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 KB 1.3 MB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 KB 1.8 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 KB 1.3 MB/s eta 0:00:001.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 1.3 MB/s eta 0:00:000:00:01


In [77]:
merged_df

,id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,director
0,19995,"Sam Worthington, Zoe Saldana, Sigourney Weaver","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"Action, Adventure, Fantasy, Science Fiction",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,James Cameron
1,285,"Johnny Depp, Orlando Bloom, Keira Knightley","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"Adventure, Fantasy, Action",http://disney.go.com/disneypictures/pirates/,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Gore Verbinski
2,206647,"Daniel Craig, Christoph Waltz, Léa Seydoux","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"Action, Adventure, Crime",http://www.sonypictures.com/movies/spectre/,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,...,2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Sam Mendes
3,49026,"Christian Bale, Michael Caine, Gary Oldman","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"Action, Crime, Drama, Thriller",http://www.thedarkknightrises.com/,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,...,2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christopher Nolan
4,49529,"Taylor Kitsch, Lynn Collins, Samantha Morton","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"Action, Adventure, Science Fiction",http://movies.disney.com/john-carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",...,2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,9367,"Carlos Gallardo, Jaime de Hoyos, Peter Marquardt","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",220000,"Action, Crime, Thriller",NaN,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,...,1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Robert Rodriguez
4799,72766,"Edward Burns, Kerry Bishé, Marsha Dietlein","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",9000,"Comedy, Romance",NaN,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,...,2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns
4800,231617,"Eric Mabius, Kristin Booth, Crystal Lowe","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...",0,"Comedy, Drama, Romance, TV Movie",http://www.hallmarkchannel.com/signedsealeddel...,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",...,2013-10-13,0,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Scott S

In [78]:
merged_df.to_csv("merged_movies.csv", index=False)
